## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import numpy as np
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,49.80,86,54,2.33,broken clouds
1,1,Panguna,PG,-6.3164,155.4848,74.41,82,78,1.14,broken clouds
2,2,Norman Wells,CA,65.2820,-126.8329,66.24,52,75,11.50,broken clouds
3,3,Awka,NG,6.2101,7.0741,73.09,95,78,5.97,light rain
4,4,Ushuaia,AR,-54.8000,-68.3000,38.86,65,0,11.50,clear sky
5,5,Kaeo,NZ,-35.1000,173.7833,53.33,98,16,5.61,few clouds
6,6,Dikson,RU,73.5069,80.5464,42.04,100,100,9.95,light rain
7,7,Hualmay,PE,-11.0964,-77.6139,63.63,78,58,12.77,broken clouds
8,8,Estelle,US,29.8458,-90.1067,91.00,65,75,6.91,broken clouds
9,9,Ribeira Grande,PT,38.5167,-28.7000,74.46,83,28,15.59,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your preferred minimum temperature for your trip? "))
max_temp = float(input("What is your preferred maximum temperature for your trip? "))

What is your preferred minimum temperature for your trip? 0
What is your preferred maximum temperature for your trip? 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_temp_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_temp_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,49.80,86,54,2.33,broken clouds
1,1,Panguna,PG,-6.3164,155.4848,74.41,82,78,1.14,broken clouds
2,2,Norman Wells,CA,65.2820,-126.8329,66.24,52,75,11.50,broken clouds
3,3,Awka,NG,6.2101,7.0741,73.09,95,78,5.97,light rain
4,4,Ushuaia,AR,-54.8000,-68.3000,38.86,65,0,11.50,clear sky
5,5,Kaeo,NZ,-35.1000,173.7833,53.33,98,16,5.61,few clouds
6,6,Dikson,RU,73.5069,80.5464,42.04,100,100,9.95,light rain
7,7,Hualmay,PE,-11.0964,-77.6139,63.63,78,58,12.77,broken clouds
8,8,Estelle,US,29.8458,-90.1067,91.00,65,75,6.91,broken clouds
9,9,Ribeira Grande,PT,38.5167,-28.7000,74.46,83,28,15.59,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_temp_df.isnull().sum()

City_ID                0
City                   0
Country                8
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_temp_df.dropna()
print(len(clean_df))
clean_df.head()

654


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,49.80,86,54,2.33,broken clouds
1,1,Panguna,PG,-6.3164,155.4848,74.41,82,78,1.14,broken clouds
2,2,Norman Wells,CA,65.2820,-126.8329,66.24,52,75,11.50,broken clouds
3,3,Awka,NG,6.2101,7.0741,73.09,95,78,5.97,light rain
4,4,Ushuaia,AR,-54.8000,-68.3000,38.86,65,0,11.50,clear sky


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hermanus,ZA,49.80,broken clouds,-34.4187,19.2345,
1,Panguna,PG,74.41,broken clouds,-6.3164,155.4848,
2,Norman Wells,CA,66.24,broken clouds,65.2820,-126.8329,
3,Awka,NG,73.09,light rain,6.2101,7.0741,
4,Ushuaia,AR,38.86,clear sky,-54.8000,-68.3000,
5,Kaeo,NZ,53.33,few clouds,-35.1000,173.7833,
6,Dikson,RU,42.04,light rain,73.5069,80.5464,
7,Hualmay,PE,63.63,broken clouds,-11.0964,-77.6139,
8,Estelle,US,91.00,broken clouds,29.8458,-90.1067,
9,Ribeira Grande,PT,74.46,light rain,38.5167,-28.7000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}" 
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found...Skipping.")
        

Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Sk

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]

clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hermanus,ZA,49.80,broken clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
2,Norman Wells,CA,66.24,broken clouds,65.2820,-126.8329,Heritage Hotel
3,Awka,NG,73.09,light rain,6.2101,7.0741,Best Western Meloch Hotel
4,Ushuaia,AR,38.86,clear sky,-54.8000,-68.3000,Albatross Hotel
5,Kaeo,NZ,53.33,few clouds,-35.1000,173.7833,Bushido Lodge


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")


In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
len(hotels["results"])

3